In [1]:
import pandas as pd
from anytree import Node, RenderTree

In [2]:
def make_map(list_child_parent):
    has_parent = set()
    all_items = {}
    
    for child, parent in list_child_parent:
        if parent not in all_items:
            all_items[parent] = {}
            
        if child not in all_items:
            all_items[child] = {}
        all_items[parent][child] = all_items[child]
        has_parent.add(child)

    result = {}
    
    for key, value in all_items.items():
        if key not in has_parent:
            result[key] = value
    
    return result

def createTree(tree_dict, root):
    for key, item in tree_dict.items():
        child = Node(key, parent=root)

        if tree_dict[key] != '':
            createTree(tree_dict[key], child)
        else:
            return

In [3]:
df = pd.read_csv('Datasets/reddit_data.csv')

# df = pd.read_csv('Datasets/sample_tweet_conversation.csv', dtype=object)

# df['id'] = df['id'].astype(str)
# df['reply_to'] = df['reply_to'].astype(str)

df = df.drop(columns=['Unnamed: 0'])
df

,user_name,id,timestamp,reply_to,comment
0,MapleViolet,hpr2kav,2021-12-24 08:55:24,rmqevj,All I know is - anyone trying to pull a fast o...
1,applescript16,hpntm2t,2021-12-23 16:24:16,rmqevj,Here’s some perspective: \n\n1) The public nat...
2,iluj13,hpnwekg,2021-12-23 17:01:54,hpntm2t,Well said. It’s only a problem if your party i...
3,forzenrose,hpnzb4r,2021-12-23 17:41:51,hpnwekg,&gt;Transparency and finding out the truth is ...
4,sec5,hpo5qkg,2021-12-23 19:10:22,hpnzb4r,'Im so done with RK Saga because it sucks to b...
...,...,...,...,...,...
177,hexquisite,hpobx5f,2021-12-23 20:25:53,rmqevj,This dum bish needs to learn from the pap fuck...
178,A-Chicken,hpr73zc,2021-12-24 09:32:25,rmqevj,"I'm sorry, but this is the opposition we're ta..."
179,PublicWar5,hpp9czp,2021-12-24 00:57:35,rmqevj,"Honestly I hate the COP, I hate how much of a ..."
180,etyn100,hpnw8qy,2021-12-23 16:59:45,rmqevj,Gotta milk some political brownies points


## Tree of Comments

In [4]:
parent = 'rmqevj'
# parent = '18831926'
root = Node(parent)

input_list = [] 

for i in range(len(df['id'].tolist())):
    if df['id'].loc[i] != df['reply_to'].loc[i]:
        input_list.append((df['id'].loc[i], df['reply_to'].loc[i]))

output_dict = make_map(input_list)
createTree(output_dict[parent], root)

In [5]:
for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))

rmqevj
├── hpr2kav
├── hpntm2t
│   ├── hpnwekg
│   │   ├── hpnzb4r
│   │   │   └── hpo5qkg
│   │   │       └── hpokj06
│   │   │           ├── hpp0qgm
│   │   │           ├── hpoqxz5
│   │   │           └── hpraj0s
│   │   │               └── hpshgjh
│   │   ├── hpo2pl5
│   │   │   └── hpo7oxl
│   │   │       └── hpo7u0x
│   │   ├── hpo76sq
│   │   ├── hpo7l9d
│   │   ├── hpradku
│   │   │   ├── hps0rop
│   │   │   └── hprqpon
│   │   ├── hpoya8b
│   │   └── hpnwve6
│   │       └── hpnz3vc
│   │           ├── hpp0vzg
│   │           └── hpo6fwl
│   │               └── hpp7l4j
│   ├── hpo1src
│   │   ├── hpolpfk
│   │   ├── hpo33t8
│   │   │   ├── hpo4que
│   │   │   │   └── hpom3uq
│   │   │   │       └── hprayhb
│   │   │   │           └── hpruuh0
│   │   │   └── hpoq8hi
│   │   │       └── hppkszp
│   │   │           └── hpppv0m
│   │   └── hpo6898
│   │       └── hpohz2e
│   ├── hpnx97j
│   │   ├── hpnxtjg
│   │   │   ├── hpo0tg0
│   │   │   │   ├── hpo1tc3
│   │   │   │   └── hprbg